Image Analysis:

    Object detection: Identify key objects in the image (e.g., "tree", "bird", "sun").
    Scene recognition: Understand the broader scene (e.g., "sunset over the ocean").
    Sentiment analysis: Derive the emotional tone of the image (e.g., "calm", "serene").

Prompter:

    Use a pre-trained GPT-2 or GPT-3 (or T5) model with prompt engineering to generate a haiku based on the extracted image features (objects, scene, sentiment).

Detector:

    Check whether the haiku generated adheres to the 5-7-5 syllable structure.
    Evaluate whether the emotional tone and meaning match the original image features.

Reinforcement Learning (RL):

    Use PPO (Proximal Policy Optimization) or any other RL algorithm to optimize the prompt and reward the haiku generation process based on structural and thematic correctness.

<H1> Fine-Tuning LLM for Haiku Structure </H1>
https://github.com/davanstrien/haiku-dpo <br>
https://github.com/glakshay/Generating-Haiku-using-GAN/tree/master/dataset


<h2> Prompter 

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained fine-tuned GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2_finetuned_haiku")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2_finetuned_haiku")

def generate_haiku_from_image(scene, sentiment, objects, model, tokenizer):
    prompt = f"Generate a haiku based on the scene: {scene}. The detected objects are {', '.join(objects)}. The emotional tone is {sentiment}."
    
    # Tokenize and generate haiku
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(inputs, max_length=50, num_return_sequences=1, temperature=0.7, top_k=50)
    
    haiku = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return haiku

# Example usage
scene = "sunset over the ocean"
sentiment = "calm and peaceful"
objects = ["tree", "bird", "sun"]

haiku = generate_haiku_from_image(scene, sentiment, objects, model, tokenizer)
print("Generated Haiku:", haiku)


<h2> Detector 

In [ ]:
import syllapy

# Reward function based on syllable accuracy and emotional tone consistency
def reward_function(poem_lines, sentiment, scene, objects, haiku, target_syllables=[5, 7, 5]):
    reward = 0

    # Syllable count reward
    syllable_violations = detect_syllable_violations(poem_lines, target_syllables)
    reward -= len(syllable_violations)  # Penalize for violations

    # Emotion check (e.g., match sentiment)
    if evaluate_emotion(haiku, sentiment[0]['label']):
        reward += 1  # Reward for matching emotion

    # Object consistency (check if key objects are referenced in the haiku)
    for obj in objects:
        if obj.lower() in haiku.lower():
            reward += 0.5  # Reward for using detected objects

    return reward


<h2> RL PPO </h2>

In [ ]:
from stable_baselines3 import PPO
import gym

class HaikuEnv(gym.Env):
    def __init__(self, model, tokenizer, scene, sentiment, objects):
        super(HaikuEnv, self).__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.scene = scene
        self.sentiment = sentiment
        self.objects = objects
        self.generated_text = ""
        self.step_count = 0

    def reset(self):
        self.generated_text = ""
        self.step_count = 0
        return self.scene

    def step(self, action):
        # Generate the next word based on action
        inputs = self.tokenizer.encode(self.generated_text, return_tensors='pt')
        outputs = self.model.generate(inputs, max_length=50, num_return_sequences=1)
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Calculate reward based on detector
        poem_lines = generated_text.split(", ")
        reward = reward_function(poem_lines, self.sentiment, self.scene, self.objects, generated_text)

        self.generated_text = generated_text
        self.step_count += 1
        done = self.step_count > 3  # Limit to 3 steps (complete haiku)
        return generated_text, reward, done, {}

# Set up PPO agent
env = HaikuEnv(model, tokenizer, scene="sunset over the ocean", sentiment="calm", objects=["tree", "bird", "sun"])
ppo = PPO("MlpPolicy", env, verbose=1)
ppo.learn(total_timesteps=10000)
